In [1]:
import os
import time
import datetime
import re
import pandas as pd


import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import chromedriver_autoinstaller
from bs4 import BeautifulSoup


startTime = time.time()
currentTime = datetime.datetime.today()
print('==============================\n')
print(f'Current time is {currentTime}')


fileDirectory = os.getcwd()
parentDirectory = os.getcwd()


class Website:
    def __init__(self, name, title, main_url, target_url):
        self.name = name
        self.title = title
        self.main_url = main_url
        self.target_url = target_url
        
    def info(self):
        print(f'Website.name : {self.name}')
        print(f'Website.title : {self.title}')
        print(f'Website.main_url : {self.main_url}')
        print(f'Website.target_url : {self.target_url}')


class WebElement:
    def __init__(self, name, type, identifier):
        self.name = name
        self.type = type
        self.identifier = identifier
        
    def info(self):
        print(f'WebElement.name : {self.name}')
        print(f'WebElement.type : {self.type}')
        print(f'WebElement.identifier : {self.identifier}')


class Preprocessor:
    def __init__(self) -> None:
        print('class Preprocessor : init')
        
    def clearText(self, texts):
        list_return = [text.text for text in texts]
        texts_cleared = [text.strip() for text in list_return]
        return texts_cleared

    def extractImage(self, contents):
        images_extracted = [tag['src'] for tag in contents]
        return images_extracted

    def extractURL(self, contents):
        url_extracted = [tag['href'] for tag in contents]
        return url_extracted
    
    def extractURLfromParent(self, contents):
        url_extracted = [tag.parent['href'] for tag in contents]
        return url_extracted

    def insertTextInURLs(self, text, location, urls):
        urls_inserted = []
        for url in urls:
            url_splited = url.split(location)
            url_inserted = url_splited[0] + location + text + url_splited[1]
            urls_inserted.append(url_inserted)
        return urls_inserted

    def preprocessContents(self, website, type, contents):
        if type == 'text':
            contents_preprocessed = self.clearText(contents)
        elif type == 'image':
            contents_preprocessed = self.extractImage(contents)
        elif type == 'url':
            if website.title == 'genre_top_100':
                contents_preprocessed = self.extractURLfromParent(contents)
            else:
                contents_preprocessed = self.extractURL(contents)
            if website.name == '블랙보드':
                contents_preprocessed = self.insertTextInURLs('courses','ultra/',contents_preprocessed)
            elif website.name == '고려대학교 경영대학':
                contents_preprocessed = [website.url.replace('notice.html','') + content for content in contents_preprocessed]
        return contents_preprocessed


class ActionChain:
    def __init__(self):
        print('class ActionChain : init')
        
    def waitAction(self, driver, action, maxWaitTime=30):
        try:
            if action.type == 'css':
                element = WebDriverWait(driver, maxWaitTime).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, action.identifier)))
            elif action.type == 'xpath':
                element = WebDriverWait(driver, maxWaitTime).until(expected_conditions.presence_of_element_located((By.XPATH, action.identifier)))
        except Exception as e:
            print('ActionChain.waitAction error :', e)

    def clickAction(self, driver, action):
        chains = webdriver.ActionChains(driver)
        if action.type == 'css':
            button = driver.find_element(By.CSS_SELECTOR, action.identifier)
        elif action.type == 'xpath':
            button = driver.find_element(By.XPATH, action.identifier)
        chains.move_to_element(button).click().perform()
        # chains.context_click(button).perform()

    def inputAction(self, driver, action):
        if action.type == 'css':
            inputElement = driver.find_element(By.CSS_SELECTOR, action.identifier)
        if action.type == 'xpath':
            inputElement = driver.find_element(By.XPATH, action.identifier)
        text =re.findall('\(([^)]+)', action.name)
        inputElement.send_keys(text)

    def enterAction(self, driver, action):
        if action.type == 'css':
            keyElement = driver.find_element(By.CSS_SELECTOR, action.identifier)
        if action.type == 'xpath':
            keyElement = driver.find_element(By.XPATH, action.identifier)
        keyElement.send_keys(Keys.ENTER)

    def doActionChain(self, driver, actions):
        for i, action in enumerate(actions):
            if 'wait' in action.name:
                self.waitAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'time' in action.name:
                driver.implicitly_wait(5)
                print(f'Step {i} : {action.name} done')
            elif 'click' in action.name:
                self.clickAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'input' in action.name:
                self.inputAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'enter' in action.name:
                self.enterAction(driver, action)
                print(f'Step {i} : {action.name} done')
            elif 'back' in action.name:
                driver.back()
                print(f'Step {i} : {action.name} done')
            else:
                print('')
            
    def doActionRoutine(self, driver, iteration, iteration_actions):
        for i, action in enumerate(iteration_actions):
            if 'wait' in action.name:
                self.waitAction(driver, action)
                # print(f'Step {i} : {action.name} done')
            elif 'load' in action.name:
                driver.implicitly_wait(5)
                # print(f'Step {i} : {action.name} done')
            elif 'time' in action.name:
                time.sleep(3)
                # print(f'Step {i} : {action.name} done')
            elif 'click' in action.name:
                webdriver.ActionChains(driver).move_to_element(iteration).click().perform()
                # print(f'Step {i} : {action.name} done')
            elif 'input' in action.name:
                self.inputAction(driver, action)
                # print(f'Step {i} : {action.name} done')
            elif 'enter' in action.name:
                self.enterAction(driver, action)
                # print(f'Step {i} : {action.name} done')
            elif 'back' in action.name:
                driver.back()
                # print(f'Step {i} : {action.name} done')
            else:
                print('')


class WebDriverUpdateChecker:
    def __init__(self):
        print('class WebDriverUpdateChecker : init')
    
    def checkWebdriverVersion(self):
        chromeVersion = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
        parentDirectoryList = os.listdir(parentDirectory)
        if 'chromedriver' not in parentDirectoryList:
            os.mkdir(parentDirectory + '/chromedriver')
        chromedriverDirectory = parentDirectory + '/chromedriver'
        chromedriverDirectoryList = os.listdir(chromedriverDirectory)
        if chromeVersion not in chromedriverDirectoryList:
            chromedriver_autoinstaller.install(False, chromedriverDirectory)
            print('\ncheckWebdriverVersion : downloaded the latest chromedriver')
        else:
            print('\ncheckWebdriverVersion : already the latest chromedriver')

    def getWebdriverDirectory(self):
        chromeVersion = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
        webdriverDirectory = parentDirectory + '/chromedriver' + '/' + chromeVersion + '/' + 'chromedriver'
        return webdriverDirectory


class Crawler(WebDriverUpdateChecker, ActionChain, Preprocessor):
    def __init__(self):
        super().__init__()
        print('\tclass Crawler : init')
    
    def getResponse(self, url):
        try:
            response = requests.get(url)
        except Exception as e:
            print('Crawler.getResponse error : ', e)
        return response

    def setChromeOptions(self, headless=True):
        chromeOptions = webdriver.ChromeOptions()
        if headless:
            chromeOptions.add_argument('headless')
        chromeOptions.add_argument('lang=ko_KR')
        chromeOptions.add_argument('disable-gpu')
        chromeOptions.add_argument('window-size=1920x1080')
        chromeOptions.add_argument(
            'user-agent='+'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36')
        return chromeOptions

    def getDriver(self, url, actions):
        super().checkWebdriverVersion()
        webdriverDirectory = super().getWebdriverDirectory()
        chromeOptions = self.setChromeOptions()
        driver = webdriver.Chrome(webdriverDirectory, options=chromeOptions)
        try:
            driver.get(url)
        except Exception as e:
            print('Crawling.getDriver error : ', e)
        if actions is not None:
            super().doActionChain(driver, actions)
        return driver

    def getSource(self, url, actions, module):
        if module == 'requests':
            reponse = self.getResponse(url)
            source = reponse.content
        elif module == 'selenium':
            driver = self.getDriver(url, actions)
            source = driver.page_source
        return source

    def parseSource(self, source, parser):
        soup = BeautifulSoup(source, parser)
        return soup

    def selectText(self, soup, content):
        if soup is not None:
            try:
                texts = soup.select(content.identifier)
            except Exception as e:
                print('Crawler.selectText error : ', e)
            return texts

    def selectTable(self, soup, content):
        if soup is not None:
            try:
                target = soup.select(content.identifier)
            except Exception as e:
                print('Crawler.selectTable error : ', e)
            table_list = pd.read_html(str(target))
            return table_list

    def selectImage(self, soup, content):
        if soup is not None:
            try:
                images = soup.find_all('img')
            except Exception as e:
                print('Crawler.selectText error : ', e)
            return images

    def getContents(self, website, contents, actions=None, module='selenium', parser='lxml'):
        source = self.getSource(website.target_url, actions, module)
        soup = self.parseSource(source, parser)
        dict_return = {}
        for content in contents:
            if 'text' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                texts = self.selectText(soup, content)
                texts_preprocessed = super().preprocessContents(website, 'text', texts)
                dict_return[key[0]] = texts_preprocessed
            elif 'table' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                table = self.selectTable(soup, content)
                dict_return[key[0]] = table[0]
            elif 'image' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                images = self.selectImage(soup, content)
                images_preprocessed = super().preprocessContents(website, 'image', images)
                dict_return[key[0]] = images_preprocessed
            elif 'url' in content.name:
                key = re.findall('\(([^)]+)', content.name)
                urls = self.selectText(soup, content)
                urls_preprocessed = super().preprocessContents(website, 'url', urls)
                dict_return[key[0]] = urls_preprocessed
            else:
                print('')
        return dict_return
    
    
    
    
    def getContentsInSequence(self, website, contents, iteration, iteration_actions_in, iteration_actions_out, actions=None, module='selenium', parser='lxml'):
        driver = self.getDriver(website.target_url, actions)
        chains = webdriver.ActionChains(driver)
        
        if iteration.type == 'css':
            iterations = driver.find_elements(By.CSS_SELECTOR, iteration.identifier)
        elif iteration.type == 'xpath':
            iterations = driver.find_elements(By.XPATH, iteration.identifier)
        keys = [a.text for a in iterations]
        
        iteration_dict = {}
        for i in range(len(iterations)):
        # for i in range(5):
            
            if iteration.type == 'css':
                iterations = driver.find_elements(By.CSS_SELECTOR, iteration.identifier)
            elif iteration.type == 'xpath':
                iterations = driver.find_elements(By.XPATH, iteration.identifier)

            super().doActionRoutine(driver, iterations[i], iteration_actions_in)
            
            source = driver.page_source
            soup = self.parseSource(source, parser)
            dict_return = {}
            for content in contents:
                if 'text' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    texts = self.selectText(soup, content)
                    texts_preprocessed = super().preprocessContents(website, 'text', texts)
                    dict_return[key[0]] = texts_preprocessed
                elif 'table' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    table = self.selectTable(soup, content)
                    dict_return[key[0]] = table[0]
                elif 'image' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    images = self.selectImage(soup, content)
                    images_preprocessed = super().preprocessContents(website, 'image', images)
                    dict_return[key[0]] = images_preprocessed
                elif 'url' in content.name:
                    key = re.findall('\(([^)]+)', content.name)
                    urls = self.selectText(soup, content)
                    urls_preprocessed = super().preprocessContents(website, 'url', urls)
                    dict_return[key[0]] = urls_preprocessed
                else:
                    print('')
            iteration_dict[keys[i]] = dict_return
            super().doActionRoutine(driver, iterations[i], iteration_actions_out)
            print(f'{website.name} - {i}/{len(iterations)} {keys[i]} Done!                                   ', end='\r')
        return iteration_dict

    
    
crawler = Crawler()

endTime = time.time()
print(f'\nRunning time : {endTime - startTime} \n')
print('==============================\n\n\n')


Current time is 2022-04-27 22:57:23.594265
class WebDriverUpdateChecker : init
	class Crawler : init

Running time : 0.0020380020141601562 






In [29]:
# Crawl {genre_names}, {genre_urls} from 'https://www.rottentomatoes.com/top'

startTime = time.time()
currentTime = datetime.datetime.today()
print('==============================\n')
print(f'Current time is {currentTime}')


rotten_tomatoes = Website('rotten tomatoes', 'genre_top_100', 'https://www.rottentomatoes.com', 'https://www.rottentomatoes.com/top')
contents = [
    WebElement('text(genre_names)', 'css', '#main_container > div.container-masonry > div:nth-child(3) > section > div > ul > li > a > div'),
    WebElement('url(genre_urls)', 'css', '#main_container > div.container-masonry > div:nth-child(3) > section > div > ul > li > a > div')
]
actions = []
genre_crawled = crawler.getContents(rotten_tomatoes, contents, module='requests')


endTime = time.time()
print(f'\nRunning time : {endTime - startTime} \n')
print('==============================\n\n\n')



for key, value in genre_crawled.items():
    print(key)
    print('{')
    print(*value, sep='\n')
    print('}')
    print()


Current time is 2022-04-28 01:43:12.844449

Running time : 5.326730012893677 




genre_names
{
Top 100 Action & Adventure Movies
Top 100 Animation Movies
Top 100 Art House & International Movies
Top 100 Classics Movies
Top 100 Comedy Movies
Top 100 Documentary Movies
Top 100 Drama Movies
Top 100 Horror Movies
Top 100 Kids & Family Movies
Top 100 Musical & Performing Arts Movies
Top 100 Mystery & Suspense Movies
Top 100 Romance Movies
Top 100 Science Fiction & Fantasy Movies
Top 100 Special Interest Movies
Top 100 Sports & Fitness Movies
Top 100 Television Movies
Top 100 Western Movies
}

genre_urls
{
/top/bestofrt/top_100_action__adventure_movies/
/top/bestofrt/top_100_animation_movies/
/top/bestofrt/top_100_art_house__international_movies/
/top/bestofrt/top_100_classics_movies/
/top/bestofrt/top_100_comedy_movies/
/top/bestofrt/top_100_documentary_movies/
/top/bestofrt/top_100_drama_movies/
/top/bestofrt/top_100_horror_movies/
/top/bestofrt/top_100_kids__family_movies/
/top/bestofrt

In [30]:
aaa = genre_crawled['genre_names'][13:]
genre_crawled['genre_names'] = aaa
genre_crawled

{'genre_names': ['Top 100 Special Interest Movies',
  'Top 100 Sports & Fitness Movies',
  'Top 100 Television Movies',
  'Top 100 Western Movies'],
 'genre_urls': ['/top/bestofrt/top_100_action__adventure_movies/',
  '/top/bestofrt/top_100_animation_movies/',
  '/top/bestofrt/top_100_art_house__international_movies/',
  '/top/bestofrt/top_100_classics_movies/',
  '/top/bestofrt/top_100_comedy_movies/',
  '/top/bestofrt/top_100_documentary_movies/',
  '/top/bestofrt/top_100_drama_movies/',
  '/top/bestofrt/top_100_horror_movies/',
  '/top/bestofrt/top_100_kids__family_movies/',
  '/top/bestofrt/top_100_musical__performing_arts_movies/',
  '/top/bestofrt/top_100_mystery__suspense_movies/',
  '/top/bestofrt/top_100_romance_movies/',
  '/top/bestofrt/top_100_science_fiction__fantasy_movies/',
  '/top/bestofrt/top_100_special_interest_movies/',
  '/top/bestofrt/top_100_sports__fitness_movies/',
  '/top/bestofrt/top_100_television_movies/',
  '/top/bestofrt/top_100_western_movies/']}

In [31]:
aaa = genre_crawled['genre_urls'][13:]
genre_crawled['genre_urls'] = aaa
genre_crawled

{'genre_names': ['Top 100 Special Interest Movies',
  'Top 100 Sports & Fitness Movies',
  'Top 100 Television Movies',
  'Top 100 Western Movies'],
 'genre_urls': ['/top/bestofrt/top_100_special_interest_movies/',
  '/top/bestofrt/top_100_sports__fitness_movies/',
  '/top/bestofrt/top_100_television_movies/',
  '/top/bestofrt/top_100_western_movies/']}

In [32]:
# Crawl {detail_label, detail_info, detail_crew, audience_reivews} from {genre_urls}

startTime = time.time()
currentTime = datetime.datetime.today()
print('==============================\n')
print(f'Current time is {currentTime}')


website_list = []
for i, genre_names in enumerate(genre_crawled['genre_names']):
    target_url = rotten_tomatoes.main_url + genre_crawled['genre_urls'][i]
    website_list.append(Website(genre_names, 'top100_movies', 'https://www.rottentomatoes.com', target_url))

table_container = {}
movies_dict = {}
for target_website in website_list:  # limit
    
    contents = [
        WebElement('table(movies_table)', 'css', '#top_movies_main > div > table')
    ]
    actions = []
    genre_table_crawled = crawler.getContents(target_website, contents, module='requests')
    table_container[target_website.name] = genre_table_crawled
    
    iteration = WebElement('iterations','css','#top_movies_main > div > table > tbody > tr > td:nth-child(3) > a')
    iteration_actions_in = [
        WebElement('click','',''),
        WebElement('load','','')
    ]
    iteration_actions_out = [
        WebElement('back','',''),
        WebElement('time','','')
    ]
    contents = [
        WebElement('text(detail_label)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-label.subtle'),
        WebElement('text(detail_info)', 'css', '#mainColumn > section.panel.panel-rt.panel-box.movie_info.media > div > div > ul > li > div.meta-value'),
        # WebElement('text(detail_crew)', 'css', '#movie-cast > div > div > div > div.media-body > a > span'),
        # WebElement('text(audience_reivews)', 'css', '#audience_reviews > ul > li > div.mop-audience-reviews__review-quote > div.mop-audience-reviews__review--comment.clamp.clamp-4.js-clamp')
    ]
    movies_top100_dict = crawler.getContentsInSequence(target_website, contents, iteration, iteration_actions_in, iteration_actions_out)
    movies_dict[target_website.name] = movies_top100_dict
    print(f'{target_website.name} -  Done!                                                                ', end='\n\n')
    
    
endTime = time.time()
print(f'\nRunning time : {endTime - startTime} \n')
print('==============================\n\n\n')




for key, value in movies_dict.items():
    print(key)
    print('{')
    print(*value, sep='\n')
    print('}')
    print()


Current time is 2022-04-28 01:43:46.594043

checkWebdriverVersion : already the latest chromedriver
Top 100 Special Interest Movies -  Done!                                                                                      ) Done!                                   


checkWebdriverVersion : already the latest chromedriver
Top 100 Sports & Fitness Movies -  Done!                                                                                                                  


checkWebdriverVersion : already the latest chromedriver
Top 100 Television Movies -  Done!                                                                                                           


checkWebdriverVersion : already the latest chromedriver
Top 100 Western Movies -  Done!                                                                                                                    


Running time : 2222.911912202835 




Top 100 Special Interest Movies
{
The Bride of Frankenstein (1935)
Fran

In [33]:
# Build DataFrame

container_genre = {}
for genre in movies_dict.keys():
    movie_titles = list(movies_dict[genre].keys())
    movie_details = list(movies_dict[genre].values())
    columns = ['Rating', 'Genre', 'Original Language', 'Director', 'Producer', 'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)', 'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix', 'Aspect Ratio', 'View the collection']

    container = []
    for i, title in enumerate(movie_titles):
        detail_label = [label.replace(':','') for label in movie_details[i]['detail_label']]
        detail_info = [info.replace('\n','').replace('  ','').replace('\xa0',' ') for info in movie_details[i]['detail_info']]
        for j, column in enumerate(columns):
            try:
                if column != detail_label[j]:
                    detail_label.insert(j, column)
                    detail_info.insert(j, None)
            except Exception as e:
                detail_info.insert(j, None)
        container.append(detail_info)
    
    df = pd.DataFrame(data=container, index=movie_titles, columns=columns)
    container_genre[genre] = df

In [34]:
# Combine DataFrame

container_table_total = {}
for genre in movies_dict.keys():
    table_a = table_container[genre]['movies_table']
    table_a.index = table_a['Title']
    table_b = container_genre[genre]
    # table_total = table_a.join(table_b)
    table_total = pd.concat([table_a, table_b], axis = 1)
    container_table_total[genre] = table_total

In [35]:
for key in list(container_table_total.keys()):
    container_table_total[key].to_csv('src/' + key + '.csv')

In [23]:
container_table_total['Top 100 Action & Adventure Movies'].to_csv('Top_100_Action_&_Adventure_Movies.csv')

In [22]:
container_table_total['Top 100 Animation Movies']

,Rank,RatingTomatometer,Title,No. of Reviews,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix,Aspect Ratio,View the collection
Toy Story 4 (2019),1.0,97%,Toy Story 4 (2019),453,G,"Fantasy, Animation, Kids & family, Adventure, ...",English,Josh Cooley,"Mark Nielsen, Jonas Rivera, Galyn Susman","Andrew Stanton, Stephany Folsom","Jun 21, 2019 wide","Oct 1, 2019",$434.0M,1h 40m,Walt Disney,"DTS, Dolby Digital, Dolby Atmos",Scope (2.35:1),Pixar
Spider-Man: Into the Spider-Verse (2018),2.0,97%,Spider-Man: Into the Spider-Verse (2018),393,PG (Mild Language|Frenetic Action Violence|The...,"Fantasy, Animation, Action, Kids & family, Adv...",English,"Bob Persichetti, Peter Ramsey, Rodney Rothman","Avi Arad, Amy Pascal, Phil Lord, Chris Miller,...","Phil Lord, Rodney Rothman","Dec 14, 2018 wide","Mar 19, 2019",$190.2M,1h 57m,"Sony Pictures Entertainment, Columbia Pictures","Dolby Atmos, DTS, Dolby Digital, SDDS",Scope (2.35:1),None
Inside Out (2015),3.0,98%,Inside Out (2015),379,PG (Some Action|Mild Thematic Elements),"Kids & family, Comedy, Fantasy, Animation",English,Pete Docter,Jonas Rivera,"Pete Docter, Meg LeFauve, Josh Cooley","Jun 19, 2015 wide","Nov 3, 2015",$2.0M,1h 35m,Walt Disney,"Dolby Atmos, Datasat",None,Pixar
Coco (2017),4.0,97%,Coco (2017),354,PG (Thematic Elements),"Adventure, Comedy, Animation, Music",English,Lee Unkrich,Darla K. Anderson,"Adrian Molina, Matthew Aldrich","Nov 22, 2017 wide","Feb 13, 2018",$210.5M,1h 49m,Walt Disney,Dolby Atmos,Scope (2.35:1),Pixar
Snow White and the Seven Dwarfs (1937),5.0,98%,Snow White and the Seven Dwarfs (1937),54,G,"Kids & family, Fantasy, Animation",English,David Hand,Walt Disney,"Ted Sears, Richard Creedon, Otto Englander, Di...",None,"Nov 16, 2004",None,1h 20m,None,Stereo,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Simpsons Movie (2007),96.0,87%,The Simpsons Movie (2007),222,PG-13 (Irreverent Humor Throughout),"Comedy, Animation",English,David Silverman,"James L. Brooks, Matt Groening, Al Jean, Mike ...","James L. Brooks, Matt Groening, Al Jean, Mike ...","Jul 27, 2007 wide","Dec 18, 2007",$183.1M,1h 26m,20th Century Fox,"Dolby SRD, DTS, SDDS",Scope (2.35:1),None
Boy and the World (O Menino e o Mundo) (2015),97.0,93%,Boy and the World (O Menino e o Mundo) (2015),58,PG (Thematic Material and Images),"Kids & family, Adventure, Animation",Portuguese (Brazil),Alê Abreu,"Fernanda Carvalho, Tita Tessler",None,"Dec 11, 2015 limited","Jul 5, 2016",$47.6K,1h 20m,GKIDS,None,None,None
The Peanuts Movie (2015),98.0,87%,The Peanuts Movie (2015),197,G,"Kids & family, Adventure, Comedy, Animation",English,Steve Martino,"Craig Schulz, Bryan Schulz, Cornelius Uliano, ...","Craig Schulz, Bryan Schulz, Cornelius Uliano","Nov 6, 2015 wide","Mar 8, 2016",$130.1M,1h 29m,20th Century Fox,Dolby Atmos,Flat (1.85:1),None
When Marnie Was There (2015),99.0,91%,When Marnie Was There (2015),100,PG (Thematic Elements|Smoking),"Drama, Fantasy, Mystery & thriller, Anime, Kid...",Japanese,Hiromasa Yonebayashi,"Yoshiaki Nishimura, Koji Hoshino","Hiromasa Yonebayashi, Keiko Niwa, Masashi Ando","May 22, 2015 limited","Oct 6, 2015",$314.3K,1h 43m,GKIDS,None,None,None
